# Photodetection data generator for the two-fluorophore system

In [ ]:
import numpy as np
from qutip import *
from random import *

In [ ]:
def gaussian(x):
    b=0
    c=0.155
    MAX=np.exp(-(0-b)**2/(2*c**2))
    
    return np.exp(-(x-b)**2/(2*c**2))/MAX/10

In [ ]:
N=2
a=destroy(N)
c1=tensor(a,qeye(N),qeye(N))
c2=tensor(qeye(N),a,qeye(N))
c3=tensor(qeye(N),qeye(N),a)

t0=0
tf=300
t_list=[t0,tf]

delta1=2
delta2=2
delta3=2
gamma1=4
gamma2=4
gamma3=2
p1=5
p2=5

nsaltos=16

nx3=11
x3_list=np.linspace(-0.6,0.6,nx3)

In [ ]:
H=delta1*c1.dag()*c1+delta2*c2.dag()*c2+delta3*c3.dag()*c3
               
c_ops=[]
c_ops.append(np.sqrt(gamma1)*c1)
c_ops.append(np.sqrt(gamma2)*c2)
c_ops.append(np.sqrt(gamma3)*c3)
c_ops.append(np.sqrt(p1)*c1.dag())
c_ops.append(np.sqrt(p2)*c2.dag())


In [ ]:
psi0=tensor(basis(N,0),basis(N,0),basis(N,0))
ntraj=100000
nu=500
nsets=int(ntraj/nu)
print(nsets)

d_list=np.linspace(0,1,nsets)

Saltos_full=np.zeros([ntraj,2,nsaltos],float)
d_full=np.zeros(ntraj,float)
   
times=[]
for h in range(2*ntraj):
    times.append([])
    
for g in range(0,nsets):
    
    d=d_list[g]
    d_full[g*nu:g*nu+nu]=d
    for k in range(0,nx3): #Bucle sobre detectores

        c_ops.append(np.sqrt(np.sqrt(gamma1*gamma3*gaussian(x3_list[k]-d/2)))*(c1+c3))
        c_ops.append(np.sqrt(np.sqrt(gamma2*gamma3*gaussian(x3_list[k]+d/2)))*(c2+c3))

        mc=mcsolve(H,psi0,t_list,c_ops,nu)

        for j in range(0,2*nu,2):    
            for i in range(0,len(mc.col_which[int(j/2)])):
                if mc.col_which[int(j/2)][i]==2: #Seleccionamos solo los saltos del detector (tercero de la lista)
                    times[2*nu*g+j].append(mc.col_times[int(j/2)][i])
                    times[2*nu*g+j+1].append(k+1)

        c_ops.pop()
        c_ops.pop()
        
        print("Detector=",k+1)
        print("Set",g+1)
    
for i in range(0,2*ntraj,2):
    matriz_paso=np.array(times[i:i+2])
    matriz_paso=matriz_paso[:, matriz_paso[0].argsort()] #Ordenar temporalmente los saltos
    matriz_paso[0]=np.diff(matriz_paso[0],prepend=0) #Transformar a time-delays
    Saltos_full[int(i/2),:,:]=matriz_paso[:,:nsaltos]  
    
np.save("Saltos_full.npy",Saltos_full)
np.save("d_full.npy",d_full)

In [ ]:
psi0=tensor(basis(N,0),basis(N,0),basis(N,0))
ntraj=10000
nu=500
nsets=int(ntraj/nu)
print(nsets)

d_list=np.linspace(0,1,nsets)

Saltos_train=np.zeros([ntraj,2,nsaltos],float)
d_train=np.zeros(ntraj,float)
   
times=[]
for h in range(0,2*ntraj):
    times.append([])
    
for g in range(0,nsets):
    
    d=d_list[g]
    d_train[g*nu:g*nu+nu]=d
    for k in range(0,nx3): #Bucle sobre detectores

        c_ops.append(np.sqrt(np.sqrt(gamma1*gamma3*gaussian(x3_list[k]-d/2)))*(c1+c3))
        c_ops.append(np.sqrt(np.sqrt(gamma2*gamma3*gaussian(x3_list[k]+d/2)))*(c2+c3))
        
        mc=mcsolve(H,psi0,t_list,c_ops,nu)

        for j in range(0,2*nu,2):
            for i in range(0,len(mc.col_which[int(j/2)])):
                if mc.col_which[int(j/2)][i]==2: #Seleccionamos solo los saltos del detector (tercero de la lista)
                    times[2*nu*g+j].append(round(mc.col_times[int(j/2)][i],2))
                    times[2*nu*g+j+1].append(k+1)

        c_ops.pop()
        c_ops.pop()
        
        print("Detector=",k+1)
        print("Set",g+1)
    
for i in range(0,2*ntraj,2):
    matriz_paso=np.array(times[i:i+2])
    matriz_paso=matriz_paso[:, matriz_paso[0].argsort()] #Ordenar temporalmente los saltos
    matriz_paso[0]=np.diff(matriz_paso[0],prepend=0) #Transformar a time-delays
    Saltos_train[int(i/2),:,:]=matriz_paso[:,:nsaltos]  
    
np.save("Saltos_train.npy",Saltos_train)
np.save("d_train.npy",d_train)